In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
import sklearn as sklearn

In [2]:
data_simple = pd.read_csv('data_simple.csv', encoding = "ISO-8859-1", low_memory=False, index_col=0)

In [70]:
#SHOULD WE BE USING ADD_DATE instead of ARREST_DATE? need to check # of missing values for ADD_DATE
data = data_simple[['BOFI_NBR','ARREST_DATE','AREG_SEQ_NBR']].copy()

In [71]:
data.count()

BOFI_NBR        280293
ARREST_DATE     150979
AREG_SEQ_NBR    280294
dtype: int64

In [72]:
#Drop rows with missing values
data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
print('%s rows' %(data.shape[0]))

150979 rows


In [73]:
#Drop if arrest date is before 1988
data = data[data['ARREST_DATE']>19880101]

In [74]:
#Drop if BOFI_NBR =/ or 00000
data = data[data['BOFI_NBR']!='/']
data = data[data['BOFI_NBR']!='00000']

In [75]:
data.count()

BOFI_NBR        147834
ARREST_DATE     147834
AREG_SEQ_NBR    147834
dtype: int64

In [78]:
#drop duplicates 
data=data.drop_duplicates()

In [79]:
data.count()

BOFI_NBR        129845
ARREST_DATE     129845
AREG_SEQ_NBR    129845
dtype: int64

In [80]:
by_bofi_nbr = data.groupby(['BOFI_NBR']).count()
by_bofi_nbr['CHECK'] = by_bofi_nbr['ARREST_DATE']-by_bofi_nbr['AREG_SEQ_NBR']

In [81]:
by_bofi_nbr[by_bofi_nbr['CHECK']!=0]

#means we can use either ARREST_DATE or AREG_SEQ_NBR to identify multiple arrests

,ARREST_DATE,AREG_SEQ_NBR,CHECK
BOFI_NBR,,,


In [82]:
#add column with # of arrests for each BOFI_NBR
data['ARREST_CNT'] = data.groupby('BOFI_NBR')['ARREST_DATE'].transform('count')

In [83]:
#FIRST_ARREST = 0 as flag for first arrest
data.sort_values(by = ['BOFI_NBR','ARREST_DATE'], inplace=True)
data['FIRST_ARREST'] = data.groupby('BOFI_NBR').cumcount()

In [92]:
#add flag for whether BOFI_NBR has rearrest
#using & for elementwise operation
data['REARREST_FLAG'] = np.where((data['ARREST_CNT']>1) & \
                                 (data['ARREST_CNT']-1 != data['FIRST_ARREST']),1,0)

In [93]:
data[:20]

,BOFI_NBR,ARREST_DATE,AREG_SEQ_NBR,ARREST_CNT,FIRST_ARREST,REARREST_FLAG
79479,013348,19900309.0,1,1.0,0,0
126493,020492,19920204.0,1,1.0,0,0
89690,100043,19900919.0,1,1.0,0,0
255206,100126,19980602.0,1,1.0,0,0
99339,100323,19910123.0,1,1.0,0,0
110723,100332,19910618.0,1,2.0,0,1
110724,100332,19940611.0,1,2.0,1,0
198075,100387,19950515.0,1,1.0,0,0
236667,100408,19970603.0,1,1.0,0,0
67470,100414,19920213.0,1,1.0,0,0
